In [ ]:
import asyncio
import json
import os

from matplotlib import pyplot as plt

import agentscope
from agentscope.agent import ReActAgent
from agentscope.embedding import (
    OpenAITextEmbedding,DashScopeMultiModalEmbedding,
)
from agentscope.formatter import OpenAIChatFormatter
from agentscope.message import Msg
from agentscope.model import OpenAIChatModel
from agentscope.rag import (
    TextReader,
    SimpleKnowledge,
    QdrantStore,
    Document,
    ImageReader,
)
from agentscope.tool import Toolkit
from config.env_loader import load_env, get_env
load_env()

In [2]:
for _ in agentscope.rag.__all__:
    if _.endswith("Reader"):
        print(f"- {_}")

- TextReader
- PDFReader
- ImageReader


In [3]:
async def example_text_reader(print_docs: bool) -> list[Document]:
    """使用 TextReader 读取文本字符串，并将文本分块为 Document 对象。"""
    # 创建 TextReader 对象
    reader = TextReader(chunk_size=512, split_by="paragraph")

    # 读取文本字符串
    documents = await reader(
        text=(
            # 我们准备一些文本数据用于演示 RAG 功能。
            "我的名字是李明，今年28岁。\n"
            "我居住在中国杭州，是一名算法工程师。我喜欢打篮球和玩游戏。\n"
            "我父亲的名字是李强，是一名医生，我的母亲是陈芳芳，是一名教师，她总是指导我学习。\n"
            "我现在在北京大学攻读博士学位，研究方向是人工智能。\n"
            "我最好的朋友是王伟，我们从小一起长大，现在他是一名律师。"
        ),
    )

    if print_docs:
        print(f"文本被分块为 {len(documents)} 个 Document 对象：")
        for idx, doc in enumerate(documents):
            print(f"Document {idx}:")
            print("\tScore: ", doc.score)
            print(
                "\tMetadata: ",
                json.dumps(doc.metadata, indent=2, ensure_ascii=False),
                "\n",
            )

    return documents


# docs = asyncio.run(example_text_reader(print_docs=True))
docs = await example_text_reader(print_docs=True)

2025-09-27 22:39:09,030 | INFO    | _text_reader:__call__:68 - Reading text with chunk_size=512, split_by=paragraph
2025-09-27 22:39:09,031 | INFO    | _text_reader:__call__:126 - Finished splitting the text into 5 chunks.


文本被分块为 5 个 Document 对象：
Document 0:
	Score:  None
	Metadata:  {
  "content": {
    "type": "text",
    "text": "我的名字是李明，今年28岁。"
  },
  "doc_id": "547e45879e96438c19ddb0f06bcf750ead3cc20e079458880f3533bcb2300a13",
  "chunk_id": 0,
  "total_chunks": 5
} 

Document 1:
	Score:  None
	Metadata:  {
  "content": {
    "type": "text",
    "text": "我居住在中国杭州，是一名算法工程师。我喜欢打篮球和玩游戏。"
  },
  "doc_id": "547e45879e96438c19ddb0f06bcf750ead3cc20e079458880f3533bcb2300a13",
  "chunk_id": 1,
  "total_chunks": 5
} 

Document 2:
	Score:  None
	Metadata:  {
  "content": {
    "type": "text",
    "text": "我父亲的名字是李强，是一名医生，我的母亲是陈芳芳，是一名教师，她总是指导我学习。"
  },
  "doc_id": "547e45879e96438c19ddb0f06bcf750ead3cc20e079458880f3533bcb2300a13",
  "chunk_id": 2,
  "total_chunks": 5
} 

Document 3:
	Score:  None
	Metadata:  {
  "content": {
    "type": "text",
    "text": "我现在在北京大学攻读博士学位，研究方向是人工智能。"
  },
  "doc_id": "547e45879e96438c19ddb0f06bcf750ead3cc20e079458880f3533bcb2300a13",
  "chunk_id": 3,
  "total_chunks": 5
} 

Docu

In [7]:
async def build_knowledge_base() -> SimpleKnowledge:
    """构建知识库。"""
    # 读取 documents 数据
    documents = await example_text_reader(print_docs=False)

    # 创建一个内存中的 Qdrant 向量存储，以及使用 DashScopeTextEmbedding 作为嵌入模型，初始化知识库
    knowledge = SimpleKnowledge(
        # 提供一个 embedding 模型用于将文本转换为向量
        embedding_model=OpenAITextEmbedding(
            api_key=get_env("OPENAI_API_KEY","SK"),
            model_name="BAAI/bge-m3",
            dimensions=1024,
        ),
        # 选择 Qdrant 作为向量存储
        embedding_store=QdrantStore(
            location=":memory:",  # 使用内存模式
            collection_name="test_collection",
            dimensions=1024,  # 嵌入向量的维度必须与嵌入模型输出的维度一致
        ),
    )

    # 将 documents 添加到知识库中
    await knowledge.add_documents(documents)

    # 从知识库中检索数据
    docs = await knowledge.retrieve(
        query="李明的父亲是谁？",
        limit=3,
        score_threshold=0.5,
    )

    print("检索到的 Document 对象：")
    for doc in docs:
        print(doc, "\n")

    return knowledge


# knowledge = asyncio.run(build_knowledge_base())
knowledge = build_knowledge_base()